In [ ]:
from google.colab import files

uploaded_weights = files.upload()

Saving annotations.txt to annotations.txt


In [ ]:
import zipfile
import os
# Шаг 3: Распаковка архива test_frames.zip
with zipfile.ZipFile("selected_pics.zip", "r") as zip_ref:
    zip_ref.extractall("selected_pics")

# Перемещение содержимого вложенной папки на уровень выше
!mv selected_pics/selected_pics/* selected_pics/

# Удаление пустой вложенной папки
!rmdir selected_pics/selected_pics
# Просмотр содержимого папки
!ls selected_pics

rmdir: failed to remove 'selected_pics/selected_pics': Directory not empty
'A001TA69_[69,625,229,708]_835.png'    'K482CB69_[53,551,187,623]_2681.png'
'A001TA69_[71,365,194,405]_835.png'    'K482CB69_[65,382,184,423]_2681.png'
'A001TA69_[73,419,205,467]_835.png'    'K482CB69_[69,352,183,390]_2681.png'
'A001TA69_[74,479,217,539]_375.png'    'K482CB69_[70,364,188,404]_2681.png'
'A001TA69_[76,425,208,476]_375.png'    'K482CB69_[75,275,176,302]_2681.png'
'A001TA69_[77,376,202,418]_375.png'    'K482PC69_[100,262,205,290]_26.png'
'A001TA69_[80,322,194,358]_375.png'    'K482PC69_[101,215,198,239]_26.png'
'A001TA69_[86,260,188,288]_835.png'    'K482PC69_[101,226,200,251]_799.png'
'A001TA69_[87,268,190,297]_375.png'    'K482PC69_[86,687,243,763]_26.png'
'A001TA69_[91,232,188,256]_835.png'    'K482PC69_[89,639,244,707]_26.png'
'A001TA69_[91,249,191,275]_375.png'    'K482PC69_[90,610,243,672]_26.png'
'A001TA69_[93,239,191,264]_835.png'    'K482PC69_[92,327,208,360]_799.png'
'A001TA69_[95,216,190,

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, LSTM, TimeDistributed, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, LSTM, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



In [ ]:
# Параметры
img_height, img_width = 64, 128
max_length = 9  # Максимальная длина российского номера (например, А123ВЕ456)


# Путь к файлу аннотаций и директории с изображениями
annotation_file = '/content/annotations.txt'
image_dir = '/content/selected_pics'
# Создание списка номеров из файла аннотаций
# Загрузка аннотаций
license_plates = []
image_names = []

with open(annotation_file, 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) == 2:
            plate, image_name = parts
            license_plates.append(plate)
            image_names.append(image_name)
        else:
            print(f"Пропущена некорректная строка: {line.strip()}")

# Загрузка изображений
image_files = os.listdir(image_dir)
images = []

for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    if os.path.isfile(image_path) and image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        try:
            img = Image.open(image_path).convert('RGB')
            img = img.resize((img_width, img_height))
            img = np.array(img) / 255.0  # Нормализация
            images.append(img)
        except Exception as e:
            print(f"Ошибка при обработке файла {image_file}: {e}")
    else:
        print(f"Пропускаем файл {image_file}, так как это не изображение или директория")

images = np.array(images)

# Проверка соответствия количества изображений и номеров
if len(images) != len(license_plates):
    raise ValueError(f"Количество загруженных изображений ({len(images)}) не соответствует количеству номеров в файле аннотаций ({len(license_plates)})")

Пропускаем файл selected_pics, так как это не изображение или директория
Пропускаем файл __MACOSX, так как это не изображение или директория
Пропускаем файл annotations.txt, так как это не изображение или директория


In [ ]:
print("Первые 5 строк файла annotations.txt:")
with open(annotation_file, 'r') as f:
    for i, line in enumerate(f):
        if i < 5:
            print(line.strip())
        else:
            break

print(f"\nОбщее количество строк в файле annotations.txt: {len(license_plates)}")
# Проверка соответствия количества изображений и номеров
if len(images) != len(license_plates):
    raise ValueError(f"Количество загруженных изображений ({len(images)}) не соответствует количеству номеров в файле аннотаций ({len(license_plates)})")
print("Проверка формата данных:")
for i in range(min(5, len(license_plates))):
    print(f"Номер: {license_plates[i]}, Изображение: {image_names[i]}")

Первые 5 строк файла annotations.txt:
H604OA69 H604OA69_[77,573,231,648]_182.png
H026CA69 H026CA69_[74,479,206,536]_4337.png
M486OX69 M486OX69_[101,275,207,299]_3783.png
T301OT69 T301OT69_[97,417,223,459]_87.png
T439CM69 T439CM69_[74,439,206,492]_2605.png

Общее количество строк в файле annotations.txt: 6512
Проверка формата данных:
Номер: H604OA69, Изображение: H604OA69_[77,573,231,648]_182.png
Номер: H026CA69, Изображение: H026CA69_[74,479,206,536]_4337.png
Номер: M486OX69, Изображение: M486OX69_[101,275,207,299]_3783.png
Номер: T301OT69, Изображение: T301OT69_[97,417,223,459]_87.png
Номер: T439CM69, Изображение: T439CM69_[74,439,206,492]_2605.png


In [ ]:
# Нахождение уникальных символов
# Нахождение уникальных символов (только русские буквы и цифры)
# Нахождение уникальных символов (только русские буквы и цифры)
unique_chars = sorted(set(''.join(license_plates)))
unique_chars = [char for char in unique_chars if char.isdigit() or (char >= 'А' and char <= 'Я')]
num_classes = len(unique_chars) + 1  # Дополнительный класс для заполнения

# Словарь для отображения символов на индексы
char_to_index = {char: index for index, char in enumerate(unique_chars)}
index_to_char = {index: char for char, index in char_to_index.items()}


In [ ]:
# Проверка нахождения уникальных символов
print("Уникальные символы:")
print(unique_chars)
print(f"Количество уникальных символов (классов): {num_classes}")

# Проверка отображения символов на индексы
print("\nОтображение символов на индексы:")
for char, index in char_to_index.items():
    print(f"'{char}': {index}")

# Проверка отображения индексов на символы
print("\nОтображение индексов на символы:")
for index, char in index_to_char.items():
    print(f"{index}: '{char}'")

# Проверка на наличие всех символов из номеров
all_chars = set(''.join(license_plates))
missing_chars = all_chars - set(unique_chars)
if missing_chars:
    print(f"\nВнимание! Следующие символы отсутствуют в уникальном наборе: {missing_chars}")
else:
    print("\nВсе символы из номеров учтены в уникальном наборе.")

# Проверка частоты встречаемости символов
from collections import Counter

char_frequency = Counter(''.join(license_plates))
print("\nЧастота встречаемости символов:")
for char, freq in char_frequency.most_common():
    print(f"'{char}': {freq}")

Уникальные символы:
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'А', 'В', 'Е', 'К', 'М', 'Н', 'О', 'Р', 'С', 'Т', 'У', 'Х']
Количество уникальных символов (классов): 23

Отображение символов на индексы:
'0': 0
'1': 1
'2': 2
'3': 3
'4': 4
'5': 5
'6': 6
'7': 7
'8': 8
'9': 9
'А': 10
'В': 11
'Е': 12
'К': 13
'М': 14
'Н': 15
'О': 16
'Р': 17
'С': 18
'Т': 19
'У': 20
'Х': 21

Отображение индексов на символы:
0: '0'
1: '1'
2: '2'
3: '3'
4: '4'
5: '5'
6: '6'
7: '7'
8: '8'
9: '9'
10: 'А'
11: 'В'
12: 'Е'
13: 'К'
14: 'М'
15: 'Н'
16: 'О'
17: 'Р'
18: 'С'
19: 'Т'
20: 'У'
21: 'Х'

Внимание! Следующие символы отсутствуют в уникальном наборе: {'P', 'H', 'M', 'A', 'Y', 'B', 'T', 'X', 'C', 'O', 'K', 'E'}

Частота встречаемости символов:
'6': 8529
'9': 8295
'C': 3157
'7': 2254
'T': 2252
'8': 2151
'0': 2100
'1': 1955
'P': 1934
'2': 1924
'3': 1843
'5': 1761
'4': 1753
'H': 1679
'Y': 1508
'O': 1480
'M': 1427
'K': 1319
'E': 1227
'A': 1186
'B': 1039
'X': 1025
'С': 54
'Т': 39
'Н': 34
'Р': 27
'М': 26
'В': 25


In [ ]:
# Проверка на наличие всех символов из номеров
all_chars = set(''.join(license_plates))
missing_chars = all_chars - set(unique_chars)
if missing_chars:
    print(f"\nВнимание! Следующие символы отсутствуют в уникальном наборе: {missing_chars}")
else:
    print("\nВсе символы из номеров учтены в уникальном наборе.")

# Проверка частоты встречаемости символов
from collections import Counter

char_frequency = Counter(''.join(license_plates))
print("\nЧастота встречаемости символов:")
for char, freq in char_frequency.most_common():
    print(f"'{char}': {freq}")


Внимание! Следующие символы отсутствуют в уникальном наборе: {'P', 'H', 'M', 'A', 'Y', 'B', 'T', 'X', 'C', 'O', 'K', 'E'}

Частота встречаемости символов:
'6': 8529
'9': 8295
'C': 3157
'7': 2254
'T': 2252
'8': 2151
'0': 2100
'1': 1955
'P': 1934
'2': 1924
'3': 1843
'5': 1761
'4': 1753
'H': 1679
'Y': 1508
'O': 1480
'M': 1427
'K': 1319
'E': 1227
'A': 1186
'B': 1039
'X': 1025
'С': 54
'Т': 39
'Н': 34
'Р': 27
'М': 26
'В': 25
'Е': 24
'О': 23
'А': 17
'У': 14
'К': 14
'Х': 6


In [ ]:
# Функция для преобразования номера в последовательность индексов
# Функция для преобразования номера в последовательность индексов
def encode_license_plate(plate):
    return [char_to_index.get(char, num_classes - 1) for char in plate]

encoded_plates = [encode_license_plate(plate) for plate in license_plates]
padded_plates = pad_sequences(encoded_plates, maxlen=max_length, padding='post', value=num_classes - 1)

In [ ]:
print("Encoded plates:")
print(encoded_plates)

print("\nPadded plates:")
print(padded_plates)

Encoded plates:
[[22, 6, 0, 4, 22, 22, 6, 9], [22, 0, 2, 6, 22, 22, 6, 9], [22, 4, 8, 6, 22, 22, 6, 9], [22, 3, 0, 1, 22, 22, 6, 9], [22, 4, 3, 9, 22, 22, 6, 9], [22, 7, 6, 1, 22, 22, 6, 9], [22, 8, 9, 7, 22, 22, 6, 9], [22, 8, 9, 7, 22, 22, 6, 9], [22, 2, 4, 2, 22, 22, 6, 9], [22, 8, 7, 6, 22, 22, 6, 9], [22, 0, 2, 6, 22, 22, 6, 9], [22, 2, 2, 9, 22, 22, 6, 9], [22, 2, 8, 2, 22, 22, 6, 9], [22, 3, 7, 3, 22, 22, 6, 9], [22, 0, 3, 6, 22, 22, 6, 9], [22, 4, 3, 7, 22, 22, 6, 9], [22, 2, 1, 9, 22, 22, 6, 9], [22, 1, 6, 2, 22, 22, 6, 9], [22, 1, 7, 6, 22, 22, 6, 9], [22, 9, 0, 2, 22, 22, 6, 9], [22, 8, 3, 8, 22, 22, 6, 9], [22, 6, 3, 4, 22, 22, 6, 9], [22, 7, 3, 6, 22, 22, 6, 9], [22, 3, 2, 6, 22, 22, 6, 9], [22, 3, 2, 2, 22, 22, 6, 9], [22, 9, 8, 7, 22, 22, 6, 9], [22, 9, 5, 6, 22, 22, 6, 9], [22, 3, 9, 7, 22, 22, 6, 9], [22, 3, 9, 1, 22, 22, 6, 9], [22, 5, 6, 2, 22, 22, 6, 9], [22, 6, 1, 9, 22, 22, 6, 9], [22, 7, 1, 9, 22, 22, 6, 9], [22, 7, 7, 9, 22, 22, 6, 9], [22, 7, 4, 2, 22, 22, 6, 9

In [ ]:
# Проверка максимального индекса
print("Проверка максимального индекса в закодированных номерах:")
max_index = np.max(padded_plates)
print(f"Максимальный индекс: {max_index}, Количество классов: {num_classes}")
if max_index >= num_classes:
    raise ValueError(f"Найден индекс {max_index}, превышающий количество классов {num_classes}")


Проверка максимального индекса в закодированных номерах:
Максимальный индекс: 22, Количество классов: 23


In [ ]:
# Разделение данных на обучающую и валидационную выборки
train_images, val_images, train_plates, val_plates = train_test_split(images, padded_plates, test_size=0.2, random_state=42)

In [ ]:
print("Максимальное значение в train_plates:", np.max(train_plates))
print("Максимальное значение в val_plates:", np.max(val_plates))
print("Уникальные значения в train_plates:", np.unique(train_plates))
print("Уникальные значения в val_plates:", np.unique(val_plates))


Максимальное значение в train_plates: 22
Максимальное значение в val_plates: 22
Уникальные значения в train_plates: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
Уникальные значения в val_plates: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]


In [ ]:
# Создание модели
# Создание модели

def create_model():
    input_shape = (img_height, img_width, 3)

    # CNN часть
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = Flatten()(x)

    # Полносвязный слой для уменьшения размерности
    x = Dense(max_length * 64, activation='relu')(x)

    # RNN часть
    x = Reshape((max_length, 64))(x)
    x = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = Dropout(0.25)(x)
    outputs = TimeDistributed(Dense(num_classes, activation='softmax'))(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# Обучение модели
# Обучение модели
def train_model(model, train_images, train_plates, val_images, val_plates, epochs=50, batch_size=32):
    print("Форма train_plates:", train_plates.shape)
    print("Форма val_plates:", val_plates.shape)

    train_plates_cat = to_categorical(train_plates, num_classes=num_classes)
    val_plates_cat = to_categorical(val_plates, num_classes=num_classes)

    print("Форма train_plates_cat:", train_plates_cat.shape)
    print("Форма val_plates_cat:", val_plates_cat.shape)
    print("Форма train_images:", train_images.shape)
    print("Форма val_images:", val_images.shape)

    # Проверка выходной формы модели
    test_output = model.predict(train_images[:1])
    print("Форма выходных данных модели:", test_output.shape)

    # Проверка соответствия форм
    if test_output.shape[1:] != train_plates_cat.shape[1:]:
        raise ValueError(f"Несоответствие форм: выход модели {test_output.shape[1:]}, целевые значения {train_plates_cat.shape[1:]}")

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()

    checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    history = model.fit(
        train_images,
        train_plates_cat,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(val_images, val_plates_cat),
        callbacks=[checkpoint, early_stopping]
    )

    return history
# Предсказание на одном изображении
def predict_on_single_image(model, image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize((img_width, img_height))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)
    predicted_indexes = np.argmax(prediction, axis=-1)[0]
    pred_plate = ''.join([index_to_char.get(index, '') for index in predicted_indexes if index < num_classes])

    print(f"Изображение: {os.path.basename(image_path)}, Предсказанный номер: {pred_plate}")

In [ ]:
model = create_model()
history = train_model(model, train_images, train_plates, val_images, val_plates)

In [ ]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Dense, LSTM, Bidirectional, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam


In [ ]:
# Параметры
img_height, img_width = 64, 256
max_length = 9  # Максимальная длина российского номера
num_classes = 23  # 26 букв + 10 цифр + 1 для пустого символа

In [ ]:
# Функция для загрузки и предобработки изображений
def load_and_preprocess_image(image_path):
    img = Image.open(image_path).convert('L')  # Преобразование в оттенки серого
    img = img.resize((img_width, img_height))
    img = np.array(img) / 255.0  # Нормализация
    return img.reshape(img_height, img_width, 1)


In [ ]:
import re
# Допустимые символы для российских номеров
russian_letters = 'АВЕКМНОРСТУХ'
digits = '0123456789'
allowed_chars = russian_letters + digits

# Пути к файлам и директориям
annotation_file = '/content/annotations.txt'
image_dir = '/content/selected_pics'



# Обновленное регулярное выражение
plate_pattern = r'^[АВЕКМНОРСТУХ]\d{3}[АВЕКМНОРСТУХ]{2}\d{2,3}$'


def encode_license_plate(plate):
    # Словарь для замены латинских букв на кириллические
    latin_to_cyrillic = {
        'A': 'А', 'B': 'В', 'E': 'Е', 'K': 'К', 'M': 'М', 'H': 'Н',
        'O': 'О', 'P': 'Р', 'C': 'С', 'T': 'Т', 'X': 'Х', 'Y': 'У'
    }

    plate = plate.upper().replace(' ', '')
    plate = ''.join(latin_to_cyrillic.get(char, char) for char in plate)

    print(f"Проверяемый номер после преобразования: {plate}")
    print(f"Допустимые буквы: {russian_letters}")

    if not re.match(plate_pattern, plate):
        print(f"Регулярное выражение не совпало: {plate_pattern}")
        raise ValueError(f"Неверный формат номерного знака: {plate}")

    encoding = np.zeros((max_length, num_classes))
    for i, char in enumerate(plate):
        if char in russian_letters:
            index = russian_letters.index(char)
        elif char.isdigit():
            index = len(russian_letters) + int(char)
        else:
            index = num_classes - 1  # Индекс для пустого символа
        encoding[i, index] = 1
    return encoding

# Пример использования
try:
    encoded = encode_license_plate("T439CM69")
    print(f"Номер T439CM69 успешно закодирован")
except ValueError as e:
    print(f"Ошибка: {e}")
print(ord(plate[0]))  # Должно быть 1058 для кириллической "Т"



Проверяемый номер после преобразования: Т439СМ69
Допустимые буквы: АВЕКМНОРСТУХ
Номер T439CM69 успешно закодирован
116


In [ ]:
# Загрузка аннотаций
license_plates = []
image_names = []

print(f"Чтение аннотаций из файла: {annotation_file}")
with open(annotation_file, 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) == 2:
            plate, image_name = parts
            license_plates.append(plate)
            image_names.append(image_name)
        else:
            print(f"Пропущена некорректная строка: {line.strip()}")

print(f"Загружено {len(license_plates)} аннотаций")

# Загрузка изображений
print(f"Загрузка изображений из директории: {image_dir}")
images = []

for image_name in image_names:
    image_path = os.path.join(image_dir, image_name)
    if os.path.isfile(image_path) and image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        try:
            img = Image.open(image_path).convert('RGB')
            img = img.resize((img_width, img_height))
            img = np.array(img) / 255.0  # Нормализация
            images.append(img)
        except Exception as e:
            print(f"Ошибка при обработке файла {image_name}: {e}")
    else:
        print(f"Файл не найден или не является изображением: {image_name}")

images = np.array(images)

# Проверка соответствия количества изображений и номеров
if len(images) != len(license_plates):
    raise ValueError(f"Количество загруженных изображений ({len(images)}) не соответствует количеству номеров в файле аннотаций ({len(license_plates)})")

print(f"Загружено {len(images)} изображений")
print(f"Форма массива изображений: {images.shape}")
print(f"Количество номеров: {len(license_plates)}")

# Пример вывода первых нескольких пар номер-изображение и их кодирование/декодирование
for i in range(min(5, len(license_plates))):
    plate = license_plates[i]
    print(f"Номер: {plate}, Изображение: {image_names[i]}")
    try:
        encoded = encode_license_plate(plate)
        decoded = decode_license_plate(encoded)
        print(f"Закодированный номер: {encoded.shape}")
        print(f"Декодированный номер: {decoded}")
        print(f"Совпадение: {'Да' if plate == decoded else 'Нет'}")
    except ValueError as e:
        print(f"Ошибка: {e}")
    print()

Чтение аннотаций из файла: /content/annotations.txt
Загружено 6512 аннотаций
Загрузка изображений из директории: /content/selected_pics
Загружено 6512 изображений
Форма массива изображений: (6512, 64, 256, 3)
Количество номеров: 6512
Номер: H604OA69, Изображение: H604OA69_[77,573,231,648]_182.png
Ошибка: Неверный формат номерного знака: H604OA69

Номер: H026CA69, Изображение: H026CA69_[74,479,206,536]_4337.png
Ошибка: Неверный формат номерного знака: H026CA69

Номер: M486OX69, Изображение: M486OX69_[101,275,207,299]_3783.png
Ошибка: Неверный формат номерного знака: M486OX69

Номер: T301OT69, Изображение: T301OT69_[97,417,223,459]_87.png
Ошибка: Неверный формат номерного знака: T301OT69

Номер: T439CM69, Изображение: T439CM69_[74,439,206,492]_2605.png
Ошибка: Неверный формат номерного знака: T439CM69



In [ ]:
# Допустимые символы для российских номеров



# Допустимые символы для российских номеров
russian_letters = 'АВЕКМНОРСТУХ'
digits = '0123456789'
allowed_chars = russian_letters + digits

# Пути к файлам и директориям
annotation_file = '/content/annotations.txt'
image_dir = '/content/selected_pics'

# Обновленное регулярное выражение
plate_pattern = r'^[АВЕКМНОРСТУХ]\d{3}[АВЕКМНОРСТУХ]{2}\d{2,3}$'

# Допустимые символы для российских номеров
russian_letters = 'АВЕКМНОРСТУХ'
digits = '0123456789'
allowed_chars = russian_letters + digits

# Регулярное выражение для проверки формата российского номера
plate_pattern = r'^[АВЕКМНОРСТУХ]\d{3}[АВЕКМНОРСТУХ]{2}\d{2,3}$'

# Словарь для замены латинских букв на кириллические
latin_to_cyrillic = {
    'A': 'А', 'B': 'В', 'E': 'Е', 'K': 'К', 'M': 'М', 'H': 'Н',
    'O': 'О', 'P': 'Р', 'C': 'С', 'T': 'Т', 'X': 'Х', 'Y': 'У'
}

def to_cyrillic(plate):
    return ''.join(latin_to_cyrillic.get(c, c) for c in plate.upper())

def encode_license_plate(plate):
    plate = to_cyrillic(plate)
    if not re.match(plate_pattern, plate):
        raise ValueError(f"Неверный формат номерного знака: {plate}")

    encoding = np.zeros((max_length, num_classes))
    for i, char in enumerate(plate):
        if char in russian_letters:
            index = russian_letters.index(char)
        elif char.isdigit():
            index = len(russian_letters) + int(char)
        encoding[i, index] = 1
    return encoding, plate

def decode_license_plate(encoding):
    plate = ''
    for char_encoding in encoding:
        if np.max(char_encoding) == 0:  # Если все нули, это конец номера
            break
        index = np.argmax(char_encoding)
        if index < len(russian_letters):
            plate += russian_letters[index]
        elif index < num_classes - 1:
            plate += str(index - len(russian_letters))
    return plate
def tokenize_plate(plate):
    if len(plate) == 8:  # Для номеров вида А123ВС45
        return [plate[0], plate[1:4], plate[4:6], plate[6:]]
    elif len(plate) == 9:  # Для номеров вида А123ВС456
        return [plate[0], plate[1:4], plate[4:6], plate[6:]]
    else:
        return list(plate)

# Загрузка аннотаций
license_plates = []
image_names = []

print(f"Чтение аннотаций из файла: {annotation_file}")
with open(annotation_file, 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) == 2:
            plate, image_name = parts
            license_plates.append(plate)
            image_names.append(image_name)
        else:
            print(f"Пропущена некорректная строка: {line.strip()}")

print(f"Загружено {len(license_plates)} аннотаций")

# Загрузка изображений
print(f"Загрузка изображений из директории: {image_dir}")
images = []

for image_name in image_names:
    image_path = os.path.join(image_dir, image_name)
    if os.path.isfile(image_path) and image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        try:
            img = Image.open(image_path).convert('L')  # Преобразование в оттенки серого
            img = img.resize((img_width, img_height))
            img = np.array(img) / 255.0  # Нормализация
            images.append(img)
        except Exception as e:
            print(f"Ошибка при обработке файла {image_name}: {e}")
    else:
        print(f"Файл не найден или не является изображением: {image_name}")


images = np.array(images)

# Проверка соответствия количества изображений и номеров
if len(images) != len(license_plates):
    raise ValueError(f"Количество загруженных изображений ({len(images)}) не соответствует количеству номеров в файле аннотаций ({len(license_plates)})")

print(f"Загружено {len(images)} изображений")
print(f"Форма массива изображений: {images.shape}")
print(f"Количество номеров: {len(license_plates)}")

# Пример использования и проверки
for i, plate in enumerate(license_plates[:5]):  # Проверяем первые 5 номеров
    try:
        cyrillic_plate = to_cyrillic(plate)
        tokens = tokenize_plate(cyrillic_plate)
        print(f"\nАннотация {i+1}:")
        print(f"Исходный номер: {plate}")
        print(f"Номер в кириллице: {cyrillic_plate}")
        print(f"Токены: {tokens}")
        encoded, encoded_plate = encode_license_plate(plate)
        print(f"Закодированный номер: {encoded.shape}")
        decoded = decode_license_plate(encoded)
        print(f"Декодированный номер: {decoded}")
        print(f"Совпадение: {'Да' if cyrillic_plate == decoded else 'Нет'}")
        print(f"Соответствующее изображение: {image_names[i]}")

        # Проверка наличия файла изображения
        image_path = os.path.join(image_dir, image_names[i])
        if os.path.exists(image_path):
            print(f"Файл изображения существует: {image_path}")
        else:
            print(f"Файл изображения не найден: {image_path}")
    except ValueError as e:
        print(f"Ошибка: {e}")
    print()

# Проверка количества файлов в директории изображений
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
print(f"\nВсего изображений в директории: {len(image_files)}")

Чтение аннотаций из файла: /content/annotations.txt
Загружено 6512 аннотаций
Загрузка изображений из директории: /content/selected_pics
Загружено 6512 изображений
Форма массива изображений: (6512, 64, 256)
Количество номеров: 6512

Аннотация 1:
Исходный номер: H604OA69
Номер в кириллице: Н604ОА69
Токены: ['Н', '604', 'ОА', '69']
Закодированный номер: (9, 23)
Декодированный номер: Н604ОА69
Совпадение: Да
Соответствующее изображение: H604OA69_[77,573,231,648]_182.png
Файл изображения существует: /content/selected_pics/H604OA69_[77,573,231,648]_182.png


Аннотация 2:
Исходный номер: H026CA69
Номер в кириллице: Н026СА69
Токены: ['Н', '026', 'СА', '69']
Закодированный номер: (9, 23)
Декодированный номер: Н026СА69
Совпадение: Да
Соответствующее изображение: H026CA69_[74,479,206,536]_4337.png
Файл изображения существует: /content/selected_pics/H026CA69_[74,479,206,536]_4337.png


Аннотация 3:
Исходный номер: M486OX69
Номер в кириллице: М486ОХ69
Токены: ['М', '486', 'ОХ', '69']
Закодированный 

In [ ]:
# Преобразование данных в numpy массивы
X = np.array(images)
y = np.array([encode_license_plate(plate)[0] for plate in license_plates])

print(f"Форма X: {X.shape}")
print(f"Форма y_plates: {y.shape}")

Форма X: (6512, 64, 256)
Форма y_plates: (6512, 9, 23)


In [ ]:
print(y[0])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (5209, 64, 256)
y_train shape: (5209, 9, 23)


In [ ]:
print(f"Форма X_train: {X_train.shape}")
print(f"Форма X_test: {X_test.shape}")
print(f"Форма y_train: {y_train.shape}")
print(f"Форма y_test: {y_test.shape}")

Форма X_train: (5209, 64, 256)
Форма X_test: (1303, 64, 256)
Форма y_train: (5209, 9, 23)
Форма y_test: (1303, 9, 23)


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Dense, LSTM, Bidirectional, TimeDistributed, BatchNormalization, Dropout,Flatten



inputs = Input(shape=(64, 256, 1))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)

x = Reshape((16, 64 * 64))(x)  # Измените первую размерность в соответствии с выходным размером MaxPooling2D

x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(32, return_sequences=True))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(207, activation='relu')(x)
outputs= Reshape((9, 23 ))(x)  # Измените первую размерность в соответствии с выходным размером MaxPooling2D



model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

batch_size = 64
epochs = 50

checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)




Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_31 (InputLayer)       [(None, 64, 256, 1)]      0         
                                                                 
 conv2d_91 (Conv2D)          (None, 64, 256, 32)       320       
                                                                 
 max_pooling2d_91 (MaxPooli  (None, 32, 128, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_92 (Conv2D)          (None, 32, 128, 64)       18496     
                                                                 
 max_pooling2d_92 (MaxPooli  (None, 16, 64, 64)        0         
 ng2D)                                                           
                                                                 
 reshape_31 (Reshape)        (None, 16, 4096)          0  

In [ ]:
# Обучение модели
history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint, early_stopping]
)

# Оценка модели на тестовых данных
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Точность на тестовых данных: {test_accuracy:.4f}")

Epoch 1/50
51/82 [=================>............] - ETA: 57s - loss: nan - accuracy: 0.1526

KeyboardInterrupt: 